<a href="https://colab.research.google.com/github/ict3104-p2-team08/ict3104-p2-team08/blob/inference_feat/3104_T08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity Detection Notebook
Welcome to the ICT3104 Team08 activity Detection notebook! You'll be taken through the steps of running a pretrained activity detection model on videos and use your own trained model.

## Prerequisites

This section provides the setting up of the environment and dependencies required.

In [ ]:
!pip install torch -f https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchaudio -f https://download.pytorch.org/whl/cu113/torchaudio-0.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchsummary==1.5.1 torchtext==0.13.1 torchvision -f https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install wandb

### Import modules

In [1]:
from ipywidgets import Dropdown, widgets
import os
from IPython.display import HTML, Video, clear_output
from base64 import b64encode
import json

## Accessing Wandb
Create an account and Login with Wandb to view analytical charts of the activity detection using the following link: https://wandb.ai/site 

In [10]:
wandb_username = widgets.Text(description='Username:')
display(wandb_username)

wandb_key = widgets.Password(
    description='Api key:',
    disabled=False)
display(wandb_key)

Text(value='', description='Username:')

Password(description='Api key:')

In [8]:
button = widgets.Button(description="Login")

display(button)

def on_button_click(b):
    !wandb login {wandb_key.value}

button.on_click(on_button_click)

Button(description='Login', style=ButtonStyle())

Output()

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\smxli/.netrc


## Toggling of pipelines

Here we will choose which feature extraction pipeline dependencies we will use between TSU, STEP and MCTCT.
The selected pipeline will be loaded authomatically whereby changes can be made at any moment in the next cell and normal execution of the following cells. 

In [11]:
current_pipeline = "TSU"
rb = widgets.RadioButtons(
    options=['TSU', 'STEP', 'MSTCT'],
    description='Pipelines:',
    disabled=False
)

def on_change(change):
    global current_pipeline
    if change['type'] == 'change' and change['name'] == 'value':
        if current_pipeline != change['new']:
            if change['new'] == "TSU":
                %cd -q ../
            elif change['new'] == "STEP":
                if os.path.basename(os.path.normpath(os.getcwd())) =="MSTCT":
                    %cd -q ..
                    %cd -q  ./STEP
                else:
                    %cd -q  ./STEP
            elif change['new'] == "MSTCT":
                if os.path.basename(os.path.normpath(os.getcwd())) =="STEP":
                    %cd -q ../
                    %cd -q ./MSTCT
                else:
                    %cd -q ./MSTCT
        current_pipeline = change['new']
        print("Current selected pipeline: {:<6}".format(current_pipeline), end='\r')

rb.observe(on_change)

display(rb)

RadioButtons(description='Pipelines:', options=('TSU', 'STEP', 'MSTCT'), value='TSU')

## Importing Datasets
More videos from the followiing link can be added using the steps below in the respective folders: https://github.com/cvdfoundation/ava-dataset

### TSU (RGB variant only)

#### Importing TSU provided Video/CSV files.

##### For video files
Step 1. Go to /ict3104-p2-team08/data/input_files/ directory
<br>
Step 2. Add in video files provided by TSU (Feature extraction/inference needed)
##### For CSV files
Step 1. Go to /ict3104-p2-team08/data/input_csv/ directory
<br>
Step 2. Add in CSV files provided by TSU (Inference needed to show actual annotated result)

#### Importing TSU provided NPY files.
Step 1. Go to /Toyota_Smarthome/pipline/data/RGB_i3d_16frames_64000_SSD/ directory
<br>
Step 2. Add in all npy files provided by TSU

#### Importing V-iashin provided RGB NPY files.
If you have already extracted the RGB NPY files do the following:
<br>
Step 1. Go to /ict3104-p2-team08/Toyota_Smarthome/pipline/data/RGB_v_iashin/ directory
<br>
Step 2. Add in the v-iashin extracted rgb npy files
<br>
If not extracted can make use of feature extraction section below (Note: feature extraction is only possible for TSU videos that can be found in <br> /ict3104-p2-team08/data/input_files/ directory)

#### Importing TSU provided Pretrained model (PDAN).
Step 1. Go to /ict3104-p2-team08/Toyota_Smarthome/pipline/models/ directory
<br>
Step 2. Add in PDAN provided by TSU

### STEP

#### Importing STEP provided Video files.

##### For video files
##### Training video files
Step 1. Go to /ict3104-p2-team08/STEP/datasets/ava/videos/train_vid/ directory
<br>
Step 2. Add in training video files provided by AVA
##### Testing video files
Step 1. Go to /ict3104-p2-team08/STEP/datasets/ava/videos/val_vid/ directory
<br>
Step 2. Add in validation video files provided by AVA

#### Importing STEP provided Pretrained model (ava_step.pth).
Step 1. Go to /ict3104-p2-team08/STEP/pretrained/ directory
<br>
Step 2. Add in ava_step.pth provided by AVA

#### To extract video frames for newly added videos (for training/testing) in /ict3104-p2-team08/STEP/datasets/ava/videos/ directory, run the below cell (Note: Run section 2 Toggling of pipelines and check STEP radio button first)

In [ ]:
%run -i ./scripts/extract_clips.py

## Data exploration

This section load and display video data from the Toyota Smarthome (TSU) project

### Jupyter -- Upload any files/datasets into the main folder

In [12]:
uploader = widgets.FileUpload(accept='.mp4',  multiple=True)
display(uploader)

FileUpload(value={}, accept='.mp4', description='Upload', multiple=True)

In [ ]:
#%cd Documents/GitHub/ict3104-p2-team08
#print(os.getcwd())

In [ ]:
#test = uploader.value
import pandas as pd

#depends on pipeline

def return_name():
    for name, file_info in uploader.value.items():
        name_file = name
    return name_file
    
    
def catch_err_before_begin():
    uploader.value
    if len(uploader.value) == 0:
        print("An exception occurred")
        print("possible reason - file too large to upload. consider manual uploading of dataset in jupyter notebook")
    else:
        return True

    
def upload_to_path(dir):
    try:
        for name, file_info in uploader.value.items():
            with open (name, 'wb') as file:
                file.write(file_info['content'])
        print("successful upload of", name, "to", dir)
    except:
        print("An exception occurred")
        
        
        
# uplaod videos to data/input_files for tsu.
# return back to root dir
if current_pipeline == "TSU":
    if catch_err_before_begin() == True:
        %cd -q  ./data/input_files
        upload_to_path("./data/input_files")
        %cd -q ../
        %cd -q ../
    

# uplaod videos to data/input_files for mstct.
# return back to mstct dir
if current_pipeline == "MSTCT":
    if catch_err_before_begin() == True:
        %cd -q ../
        %cd -q  ./data/input_files
        upload_to_path("./data/input_files")
        %cd -q ../
        %cd -q ../
        %cd -q ./MSTCT
    
            
if current_pipeline == "STEP":
    #check if trainval or test
    xl = pd.ExcelFile("./datasets/ava/check.xlsx")
    df = xl.parse("Sheet1")
    
    if catch_err_before_begin() == True:
        #name is not working
        if return_name() in df.values:
            print(name,"exists in Dataframe")
            
            row = df.loc[df['name'] == name]
            get_type_trainval = row.loc[:,"type"] == "trainval"

            #if video belongs to trainval
            if get_type_trainval.bool() == True: 
                %cd -q  ./datasets/ava/videos/trainval
                upload_to_path("./STEP/dataset/ava/videos/trainval")
            else:
                %cd -q  ./datasets/ava/videos/test
                upload_to_path("./STEP/dataset/ava/videos/test")
            %cd -q ../
            %cd -q ../
            %cd -q ../
            %cd -q ../
        else:
            print(" video file does not belong to STEP")

In [ ]:
file_list = []
if current_pipeline == "TSU":
    FOLDER_PATH = 'input_files'
    ROOT_PATH = './data'

    file_list = os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH))
elif current_pipeline == "STEP":
    for root, dirs, files in os.walk("./datasets/ava/videos"):
        for file in files:
            #append the file name to the list
            file_list.append(file)
            
elif current_pipeline == "MSTCT":
    #change to whatever is needed
    ROOT_PATH = './dataset'
    file_list = os.listdir(os.path.join(ROOT_PATH))

w = widgets.Dropdown(
    options= file_list,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']

w.observe(on_change)

display(w)

In [ ]:
if current_pipeline == "TSU":
    video_path = './data/input_files/%' + w.value
elif current_pipeline == "STEP":
    if os.path.isfile('./datasets/ava/videos/train_vid/' + w.value):
        video_path = './datasets/ava/videos/train_vid/' + w.value
    elif os.path.isfile('./datasets/ava/videos/val_vid/' + w.value):
        video_path = './datasets/ava/videos/val_vid/' + w.value

locate= ""

for i in video_path: 
    if i=='%':
        pass
    else:
        locate+=i 

def show_video(locate, video_width = 500):
    video_file = open(locate, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
  
#Video(locate, embed=True, width=320, height=320)
show_video(locate)

## Feature Extraction (TSU)

This section allows extracting of features using TSU pipeline by selecting the video from the list loaded in the file. 

### Feature extraction prequisite 

In [13]:
#function to generate training and testing lists base on cs json file
def unextracted_list():
    full_vid_list = "./Toyota_Smarthome/pipline/data/smarthome_CS_51.json"
    full_vid_file = open(full_vid_list)
    full_vid_json = json.load(full_vid_file)
    
    unextracted_vid_list = []
    for vid_key in full_vid_json:
        unextracted_vid_list.append(vid_key)
        
    extracted_vid_list = "./Toyota_Smarthome/pipline/data/i3d_CS.json"
    extracted_vid_file = open(extracted_vid_list)
    extracted_vid_json = json.load(extracted_vid_file)
    
    for extracted_vid_key in extracted_vid_json:
        if extracted_vid_key[:-4] in unextracted_vid_list:
            unextracted_vid_list.remove(extracted_vid_key[:-4])
    unextracted_vid_list.sort()
    return unextracted_vid_list

### Select video to extract feature

In [14]:
unextracted_vid_sel = widgets.SelectMultiple(
options=unextracted_list(),
rows=10,
description='Videos:',
disabled=False
)

display(unextracted_vid_sel)

selected_videos = []

extract_but = widgets.Button(description = 'Extract')
def on_ext_button_clicked(b):
    combine_dataset = unextracted_vid_sel.value
    selected_videos = list(combine_dataset)
    selected_videos_string=(','.join(selected_videos))
    #extract features as npy
    %cd -q video_features
    %pwd
    %run -i ./test.py -videosToExtract={selected_videos_string}
    %cd -q ..


extract_but.on_click(on_ext_button_clicked)

display(extract_but)

SelectMultiple(description='Videos:', options=('P02T02C03', 'P02T02C06', 'P02T02C07', 'P02T03C07', 'P02T04C04'…

Button(description='Extract', style=ButtonStyle())

checking for selected video's npy


AssertionError: Torch not compiled with CUDA enabled

## Inference with pretrained model

This section allows generation of caption videos which can also be done on models you have trained which would automatically appear in the dropdown input below. To generate the videos, follow these steps: 
* Select a pre-trained model 
* Choose a video input
* generate video

### Select a pre-trained model

In [15]:
if current_pipeline == "TSU":
    MODEL_PATH = 'models'
    MODEL_ROOT_PATH = './Toyota_Smarthome/pipline/'

    file_list = os.listdir(os.path.join(MODEL_ROOT_PATH, MODEL_PATH))
elif current_pipeline == "STEP":
    MODEL_PATH = ''
    MODEL_ROOT_PATH = './pretrained'

    file_list = os.listdir(os.path.join(MODEL_ROOT_PATH, MODEL_PATH))

model_dropdown = widgets.Dropdown(
    options=file_list,
    description='Model:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']

model_dropdown.observe(on_change)
display(model_dropdown)

Dropdown(description='Model:', options=('PDAN',), value='PDAN')

### Choose an input video

In [16]:
VIDEO_PATH = 'input_files'

model_name = model_dropdown.value
    
#JSON_FILEPATH = "./Toyota_Smarthome/pipline/model_videos.json"
#f = open(JSON_FILEPATH)
#data = json.load(f)
#testable_video = data[model_name]
#testable_video = [video + ".mp4" for video in testable_video]
#print(testable_video)

if current_pipeline == "TSU":
    VIDEO_ROOT_PATH = './data/'
    full_testable_video = os.listdir(os.path.join(VIDEO_ROOT_PATH, VIDEO_PATH))
    
    if model_name == "PDAN":
        file_list = full_testable_video
    else:
        JSON_FILEPATH = "./Toyota_Smarthome/pipline/data/" + model_name + "_CS.json"
        f = open(JSON_FILEPATH)
        data = json.load(f)
        testable_video = []
        for video_key, video_values in data.items():
            if video_values["subset"] == "testing":
                testable_video.append(video_key)
        file_list = testable_video
elif current_pipeline == "STEP":
    full_testable_video = os.listdir(os.path.join('../data/', VIDEO_PATH))
    file_list = full_testable_video

video_dropdown = widgets.Dropdown(
    options=file_list,
    description='Video:',
    disabled=False,
)


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']

video_dropdown.observe(on_change)
display(video_dropdown)

Dropdown(description='Video:', options=('P02T01C06.mp4', 'P02T01C07.mp4', 'P02T02C03.mp4', 'P02T02C06.mp4', 'P…

In [17]:
if current_pipeline == "TSU":
    video_selected = video_dropdown.value.replace('.mp4','')
    model_selected = model_dropdown.value
    model_path = "./Toyota_Smarthome/pipline/models/" + model_selected
    %run -i ./Toyota_Smarthome/pipline/test2.py -videofile={video_selected} -load_model={model_path} -name={model_selected}
    video_path = "./Toyota_Smarthome/pipline/video_output/" + video_selected + "_caption.mp4"
elif current_pipeline == "STEP":
    %run -i ./extract_frames.py --video_name={video_dropdown.value}
    %run -i ./demo.py --model_name={model_dropdown.value} --video_name={video_dropdown.value}
    video_path = "./video_output/" + video_dropdown.value
Video(video_path, embed=True)

100%|██████████| 536/536 [00:04<00:00, 111.70it/s]


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## Training of new model

This section allows the training/testing of models of your own creation. Follow the steps below to train your own model starting the training prerequisite. 

### Training prerequisite

In [18]:
#function to generate training and testing lists base on cs json file
def training_testing_list():
    split_setting = "CS"
    cs_filepath = "./Toyota_Smarthome/pipline/data/i3d_CS.json"
    if split_setting == "CS":
        split_setting_file = open(cs_filepath)
        split_setting_json = json.load(split_setting_file)
    training_list = []
    testing_list = []
    for video_key, video_data in split_setting_json.items():
        if video_data["subset"] == "training":
            training_list.append(video_key[:-4] + ".mp4")
        elif video_data["subset"] == "testing":
            testing_list.append(video_key[:-4] + ".mp4")
    #sort lists according to naming
    training_list.sort()
    testing_list.sort()
    return training_list, testing_list

### Choose datasets for training/testing

In [19]:
if current_pipeline == "TSU":
    training_list, testing_list = training_testing_list()
elif current_pipeline == "STEP":
    training_list = os.listdir(os.path.join("./datasets/ava/videos/", "train_vid"))
    testing_list = os.listdir(os.path.join("./datasets/ava/videos/", "val_vid"))


training_box = widgets.BoundedIntText(value=1, min=1, max=100)
testing_box = widgets.BoundedIntText(value=1, min=1, max=100)

tot_train_vid = len(training_list)
tot_test_vid = len(testing_list)

training_label = widgets.Label(value="% of Training video (Total: " + str(tot_train_vid) + ")")
testing_label = widgets.Label(value="% of Testing video (Total: " + str(tot_test_vid) + ")")

display(training_label)
display(training_box)
display(testing_label)
display(testing_box)

###new

selected_videos = []

but = widgets.Button(description = 'Enter')
if current_pipeline == "TSU":
    def on_button_clicked(b):
        global selected_videos_string
        sel_num_train = int((training_box.value/100) * tot_train_vid)
        if sel_num_train < 1:
            sel_num_train = 1
        sel_num_test = int((testing_box.value/100) * tot_test_vid)
        if sel_num_test < 1:
            sel_num_test = 1
        train_list = training_list[0:sel_num_train]
        test_list = testing_list[0:sel_num_test]
        combine_dataset = train_list + test_list
        selected_videos = list(combine_dataset)
        for i in range(len(selected_videos)):
            selected_videos[i] = selected_videos[i].replace('.mp4','')
        selected_videos_string=(','.join(selected_videos))
elif current_pipeline == "STEP":
    def on_button_clicked(b):
        global selected_train_videos
        global selected_test_videos
        sel_num_train = int((training_box.value/100) * tot_train_vid)
        if sel_num_train < 1:
            sel_num_train = 1
        sel_num_test = int((testing_box.value/100) * tot_test_vid)
        if sel_num_test < 1:
            sel_num_test = 1
        train_list = training_list[0:sel_num_train]
        test_list = testing_list[0:sel_num_test]
        for i in range(len(train_list)):
            train_list[i] = train_list[i].replace('.mp4','')
            train_list[i] = train_list[i].replace('.mkv','')
            train_list[i] = train_list[i].replace('.webm','')
        for i in range(len(test_list)):
            test_list[i] = test_list[i].replace('.mp4','')
            test_list[i] = test_list[i].replace('.mkv','')
            test_list[i] = test_list[i].replace('.webm','')
        selected_train_videos=(','.join(train_list))
        selected_test_videos=(','.join(test_list))

but.on_click(on_button_clicked)

display(but)

Label(value='% of Training video (Total: 1)')

BoundedIntText(value=1, min=1)

Label(value='% of Training video (Total: 3)')

BoundedIntText(value=1, min=1)

Button(description='Enter', style=ButtonStyle())

### Model name, Batch size, Epochs
After training your model, you will be able to use it in the generation of features on a selected video of your choice. Thus, naming your model is crucial. 

**Do note that the maximum batch size is 4 and epochs is 1000.** 

In [20]:
inp = widgets.Text(description='Model name:')
batch_size = widgets.BoundedIntText(description='Batch size:',
                                   value=2,
                                   max=4)
epochs = widgets.BoundedIntText(description='Epochs:',
                               value=1000,
                               max=1000)
learning_rate = widgets.BoundedIntText(value=1)
button = widgets.Button(description="Enter")

button = widgets.Button(description="Enter")

def on_button_clicked(b):
    inp.value
    batch_size.value
    epochs.value
    learning_rate.value

button.on_click(on_button_clicked)
display(inp)
display(batch_size)
display(epochs)
display(widgets.HBox([widgets.Label(value="Learning Rate:"), learning_rate]))
display(button)

Text(value='', description='Model name:')

BoundedIntText(value=2, description='Batch size:', max=4)

BoundedIntText(value=1000, description='Epochs:', max=1000)

Button(description='Enter', style=ButtonStyle())

### Start training model

In [ ]:
global current_pipeline

model_name = inp.value
epochValue = (epochs.value)
batchSize = str(batch_size.value)
lrValue = learning_rate.value

if current_pipeline == "TSU":
    %run -i ./Toyota_Smarthome/pipline/train.py -video_train_test={selected_videos_string} -model=PDAN -name={model_name} -batch_size={batchSize} -epoch={epochValue} -lr={lrValue}
    %wandb {wandb_username.value}/ICT3104_project
elif current_pipeline == "STEP":
    %run -i ./overwrite_csv_trainval.py --video_train={selected_train_videos} --video_test={selected_test_videos}
    %run -i ./scripts/generate_label.py ./datasets/ava/label/new_train.csv
    %run -i ./scripts/generate_label.py ./datasets/ava/label/new_val.csv
    %run -i ./train.py --model_name={model_name} --batch_size={batchSize} --max_epochs={epochValue} --base_lr={lrValue}
    %wandb {wandb_username.value}/ICT3104_project_STEP